In [ ]:
!pip install --upgrade pip >> /dev/null
!pip install -U --pre efficientnet >> /dev/null
!pip install tensorflow-addons

import tensorflow as tf
#import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution

#disable_eager_execution()
#tf.enable_eager_execution()
import efficientnet.tfkeras as efn
import tensorflow_addons as tfa

In [ ]:
import pandas as pd
import numpy as np
import gc # garbage collection
from kaggle_datasets import KaggleDatasets
import re, math
import tensorflow.keras.backend as K
from tensorflow.python.keras.utils import losses_utils
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import pickle
import random as r
import cv2, os


In [ ]:
DEVICE = "TPU" # or "GPU"
SEED = 42 # USE DIFFERENT SEED FOR DIFFERENT STRATIFIED KFOLD
FOLDS = 5 # NUMBER OF FOLDS. USE 3, 5, OR 15 
IMG_SIZES = [384] * FOLDS
BATCH_SIZES = [32] * FOLDS
EPOCHS = [40] * FOLDS
EFF_NETS = [5] * FOLDS
WGTS = [1 / FOLDS] * FOLDS # WEIGHTS FOR FOLD MODELS WHEN PREDICTING TEST
TTA = 15 # TEST TIME AUGMENTATION FACTOR
NUMOFCLASSES = 7
CLASSWEIGHTS = {0:1.28545758 , 1: 0.21338021,2:2.78349083 ,3:4.37527304 ,4:1.30183284,5:12.44099379,6:10.07545272} 


In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
import cv2, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline
train = pd.read_csv('/kaggle/input/ham10k-withtest-stratified5fold-inpainted384x384/Ham10k_384_Inpainted_5FoldStratified/HAM10000_Train_Edited.csv')


In [ ]:
GCS_PATH  = [None]*FOLDS
for i,k in enumerate(IMG_SIZES):
    GCS_PATH[i] = KaggleDatasets().get_gcs_path('ham10k-minimal')
#     GCS_PATH[i] = KaggleDatasets().get_gcs_path('isic-2019-train-384x384-inpainted-stratified-tfrec')

# files_train = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/stratified_jpg_384_inpainted_tfrec/train*.tfrec')))
# files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/stratified_jpg_384_inpainted_tfrec/train04-5062.tfrec')))
#files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/test*.tfrec')))
files_train = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/train*.tfrec')))
files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/test*.tfrec')))


In [ ]:
print(GCS_PATH)

In [ ]:
files_train

In [ ]:
ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, DIM=256):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:




def read_labeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        'target'                       : tf.io.FixedLenFeature([], tf.string),
        'target_id'                    : tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    label = tf.cast(example['target_id'], tf.int64)
    #print("Example: ",example,"Done")
    return (example['image'], label)

def read_unlabeled_tfrecord(example , return_image_name):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        'target'                       : tf.io.FixedLenFeature([], tf.string),
        'target_id'                    : tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    #label = tf.cast(example['multitarget_label'], tf.int32)
    return (example['image'], example['image_name'] if return_image_name else 0)

def read_test_unlabeled_tfrecord(example, return_image_name):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string)
       # 'patient_id'                   : tf.io.FixedLenFeature([], tf.int64),
       # 'sex'                          : tf.io.FixedLenFeature([], tf.int64),
       # 'age_approx'                   : tf.io.FixedLenFeature([], tf.int64),
       # 'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
       # 'width'                        : tf.io.FixedLenFeature([], tf.int64),
       # 'height'                       : tf.io.FixedLenFeature([], tf.int64)
        
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return (example['image'], example['image_name'] if return_image_name else 0)


 
def prepare_image(img, augment=True, dim=384):   
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
#     img = tf.image.per_image_standardization(img)    
  
    if augment:
        img = transform(img,DIM=dim)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.7, 1.3)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        #img = sprinkles(img)
        #img = microscopicCutOut(img)

    img = tf.reshape(img, [dim, dim, 3])
    return img

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files
    sum=0
    for filename in filenames:
        n = filename.split('-')[-1]
        n = n.split('.')[0]
        sum = sum + int(n)
    return sum
    #n = [int(re.compile(r"-([0-9]*).").search(filename).group(1)) for filename in filenames]
    #return np.sum(n)
    
def get_dataset(files, augment = False, shuffle = False, repeat = False, 
                labeled=True,return_image_names=True, batch_size=8, dim=384):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        #ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
        ds = ds.map(lambda example: read_labeled_tfrecord(example), 
                    num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_names), 
                    num_parallel_calls=AUTO)      
    #print("Going to map againQQQQQQQQQQQQQQQQQQQQQ")
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, dim=dim), 
                                               imgname_or_label), 
                num_parallel_calls=AUTO)
    
    #print("DSS outside: ",ds)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds

def get_test_dataset(files, augment = False, shuffle = False, repeat = False, 
                labeled=True,return_image_names=True, batch_size=8, dim=384):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    #if labeled: 
    #    ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
        #ds = ds.map(lambda example: read_labeled_tfrecord(example, return_target), 
                    #num_parallel_calls=AUTO)
    #else:
    ds = ds.map(lambda example: read_test_unlabeled_tfrecord(example, return_image_names), 
                    num_parallel_calls=AUTO)      
    
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, dim=dim), 
                                               imgname_or_label), 
                num_parallel_calls=AUTO)
    
    #print("Printing ds : " ,ds)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds


In [ ]:
#import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Add, Dense
from tensorflow.keras.models import Model

def our_cnn(dim=128, num_classes=7):   
    input_shape = (dim, dim, 3)
    num_classes = 7

    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Create the model
    #model = Model(inputs=inputs, outputs=x)
    opti = tfa.optimizers.RectifiedAdam(lr=0.00032, total_steps=10000,
                               warmup_proportion=0.1, min_lr=1e-7)
    #loss = tfa.losses.SigmoidFocalCrossEntropy(alpha=0.90,gamma=2.0,
                                    #reduction=losses_utils.ReductionV2.NONE)
    loss= tf.keras.losses.SparseCategoricalCrossentropy()
    #METRICS = [tf.keras.metrics.SparseCategoricalAccuracy(), tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='SEN/Recall'),'accuracy']
    METRICS = [tf.keras.metrics.SparseCategoricalAccuracy()]
    model.compile(optimizer=opti, loss=loss, metrics=METRICS)
    print(model.summary())
#     model.summary()
    

    return model

In [ ]:
def get_lr_callback(batch_size=8):    
    lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_acc', factor=0.3, patience=2, verbose=2,
        mode='max', min_delta=0.001, min_lr=0.000000001
    )

    return lr_callback

In [ ]:
VERBOSE = 1
DISPLAY_PLOT = False

skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)

oof_pred = []; oof_tar = []; oof_val = []; oof_names = []; oof_folds = []
preds = np.zeros((count_data_items(files_test),NUMOFCLASSES))
preds1 = np.zeros((count_data_items(files_test),NUMOFCLASSES))
pred_foldWise = np.asarray([preds]*FOLDS)

In [ ]:
print(preds1)

In [ ]:
preds.shape

In [ ]:
for fold,(idxT,idxV) in enumerate(skf.split(np.arange(5))): 
    print(fold,((idxT,idxV)))

In [ ]:
print(GCS_PATH[0])

In [ ]:
CLASSES = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']

In [ ]:
for fold,(idxT,idxV) in enumerate(skf.split(np.arange(5))):    
    if DEVICE=='TPU':
        if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    print('#'*25); print('#### FOLD',fold+1)
    print('#### Image Size %i and batch_size %i'%
          (IMG_SIZES[fold], BATCH_SIZES[fold]*REPLICAS))

    files_train = tf.io.gfile.glob([GCS_PATH[fold] + '/train0%.1i*.tfrec'%x for x in idxT])

    np.random.shuffle(files_train); print('#'*25)
    files_valid = tf.io.gfile.glob([GCS_PATH[fold] + '/train0%.1i*.tfrec'%x for x in idxV])
    files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[fold] + '/test*.tfrec')))
    #files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[fold] + '/test*.tfrec')))
    #files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[fold] + '/stratified_jpg_384_inpainted_tfrec/train04-5062.tfrec')))

    # SAVE BEST MODEL EACH FOLD
    sv = tf.keras.callbacks.ModelCheckpoint(
        'cuswfold-%i.h5'%fold, monitor='val_sparse_categorical_accuracy', verbose=0, save_best_only=True,
        save_weights_only=True, mode='max', save_freq='epoch')
    
    # early stopping with 5 patience
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_sparse_categorical_accuracy', mode = 'max', patience = 5, 
                          verbose = 2, min_delta = 0.0001, restore_best_weights = True)

    a = int(count_data_items(files_train)); b=int(count_data_items(files_valid));
    print('AUGMENTED TRAIN SIZE: ', a, ' * ', TTA, ' = ', a*TTA)
    print('AUGMENTED VALID SIZE: ', b, ' * ', TTA, ' = ', b*TTA)
    
    # BUILD MODEL
    K.clear_session()
    with strategy.scope():
        # model = get_InceptionResNetV2(dim=IMG_SIZES[fold])
        # model = get_DenseNet121(dim=IMG_SIZES[fold])
        #model = get_EFF_NET(dim=IMG_SIZES[fold], ef=7, output_bias = None)
        model = our_cnn(384,NUMOFCLASSES)
   
    # TRAIN
    print('Training...')
    history = model.fit(
        get_dataset(files_train, augment=True, shuffle=True, repeat=True, 
                    dim=IMG_SIZES[fold], batch_size = BATCH_SIZES[fold]),
        epochs = 60,
        #epochs = 3,
        callbacks = [sv, early_stopping, get_lr_callback(BATCH_SIZES[fold])], 
        steps_per_epoch = count_data_items(files_train)/BATCH_SIZES[fold]//REPLICAS,
        validation_data = get_dataset(files_valid,augment=False,shuffle=False, repeat=False,
                                      dim=IMG_SIZES[fold]),
        verbose = VERBOSE,
        class_weight=CLASSWEIGHTS
    )

    print('Loading best model...')
    model.load_weights('cuswfold-%i.h5'%fold)
    
    # PREDICT OOF USING TTA
    print('Predicting OOF with TTA...')
    ds_valid = get_dataset(files_valid,labeled=False,return_image_names=False,augment=True,
            repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
    ct_valid = (count_data_items(files_valid)); STEPS = TTA * ct_valid/BATCH_SIZES[fold]/4/REPLICAS
    pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,]
    
    oof_pred.append( np.mean(pred.reshape((ct_valid,TTA,NUMOFCLASSES),order='F'),axis=1))
#     oof_pred.append( np.mean(pred.reshape((ct_valid,TTA,NUMOFCLASSES),order='F'),axis=1))
    
    # GET OOF TARGETS AND NAMES
    ds_valid = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold],
            labeled=True, return_image_names=True)
    oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )
    oof_folds.append( np.ones_like(oof_tar[-1],dtype='int8')*fold )
    ds = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold],
                labeled=False, return_image_names=True)
    oof_names.append( np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())]))
    
#    auc = roc_auc_score(oof_tar[-1],oof_pred[-1])
#    oof_val.append(np.max( history.history['val_sparse_categorical_accuracy'] ))
#    #print('#### FOLD %i OOF AUC without TTA = %.4f, with TTA = %.4f'%(fold+1,oof_val[-1],auc))

     # PREDICT TEST with TTA
    print('Predicting Test with TTA...')
    #ds_test = get_test_dataset(files_test,labeled=False,return_image_names=False,augment=True,
    #        repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
    ds_test = get_test_dataset(files_test,labeled=False,return_image_names=False,augment=True,
            repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
    ct_test = count_data_items(files_test); STEPS = TTA * ct_test/BATCH_SIZES[fold]/4/REPLICAS
    pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:TTA*ct_test,]
    tmp_pred = np.mean(pred.reshape((ct_test,TTA,NUMOFCLASSES),order='F'),axis=1)
    preds += tmp_pred * WGTS[fold]
    
    # PREDICT TEST without TTA
    print('Predicting Test without TTA...')
    #ds_test = get_test_dataset(files_test,labeled=False,return_image_names=False,augment=False,
    #        repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
    ds_test = get_test_dataset(files_test,labeled=False,return_image_names=False,augment=False,
            repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
    ct_test = count_data_items(files_test); STEPS = 1 * ct_test/BATCH_SIZES[fold]/4/REPLICAS
    pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:1*ct_test,]
    tmp_pred1 = np.mean(pred.reshape((ct_test,1,NUMOFCLASSES),order='F'),axis=1)
    preds1 += tmp_pred1 * WGTS[fold]

    pred_foldWise[fold] += tmp_pred
    
    hist=dict(zip(list(history.history.keys()), np.array(list(history.history.values()))))
    pickle.dump(hist, open("cuswhistory_fold-%i.p"%(fold+1), "wb"))
    # PLOT TRAINING
    if DISPLAY_PLOT:
        plt.figure(figsize=(15,5))
        plt.plot(np.arange(EPOCHS[fold]),history.history['sparse_categorical_accuracy'],'-o',label='Train sparse_categorical_accuracy',color='#ff7f0e')
        plt.plot(np.arange(EPOCHS[fold]),history.history['val_sparse_categorical_accuracy'],'-o',label='Val val_sparse_categorical_accuracy',color='#1f77b4')
        x = np.argmax( history.history['val_sparse_categorical_accuracy'] ); y = np.max( history.history['val_sparse_categorical_accuracy'] )
        xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max sparse_categorical_accuracy\n%.4f'%y,size=14)
        plt.ylabel('AUC',size=14); plt.xlabel('Epoch',size=14)
        plt.legend(loc=2)
        plt2 = plt.gca().twinx()
        plt2.plot(np.arange(EPOCHS[fold]),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
        plt2.plot(np.arange(EPOCHS[fold]),history.history['val_loss'],'-o',label='Val Loss',color='#1f77b4')
        x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
        ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
        plt.ylabel('Loss',size=14)
        plt.title('FOLD %i - Image Size %i, EFFNET B%i'% (fold+1,IMG_SIZES[fold],EFF_NETS[fold]),size=18)
        plt.legend(loc=3)
        plt.show()

    del model; z = gc.collect()
    

In [ ]:
# COMPUTE OVERALL OOF AUC
oof = np.concatenate(oof_pred); true = np.concatenate(oof_tar);
names = np.concatenate(oof_names); folds = np.concatenate(oof_folds)
auc = roc_auc_score(true,oof, multi_class='ovr')
print('Overall OOF AUC with TTA = %.4f'%auc)

# SAVE OOF TO DISK
df_oof = pd.DataFrame(dict(
    image_name = names, target=true, targetA=oof[:,0], targetB=oof[:,1], targetC=oof[:,2], targetD=oof[:,3], targetE=oof[:,4], targetF=oof[:,5], targetG=oof[:,6], fold=folds))
df_oof.to_csv('cuswoof.csv',index=False)
# df_oof.head()

In [ ]:
len(oof[:,0])

In [ ]:
#ds = get_test_dataset(files_test, augment=False, repeat=False, dim=IMG_SIZES[fold],
#                 labeled=False, return_image_names=True)
ds = get_test_dataset(files_test, augment=False, repeat=False, dim=IMG_SIZES[fold],
                 labeled=False, return_image_names=True)

image_names = np.array([img_name.numpy().decode("utf-8")
                        for img, img_name in iter(ds.unbatch())])

In [ ]:
submission = pd.DataFrame(dict(image_name=image_names, targetA=preds1[:,0], targetB=preds1[:,1], targetC=preds1[:,2], targetD=preds1[:,3], targetE=preds1[:,4], targetF=preds1[:,5], targetG=preds1[:,6]))
submission = submission.sort_values('image_name')
submission.to_csv('cuswwithout_TTA_submission.csv', index=False)
submission.head()

In [ ]:
submission = pd.DataFrame(dict(image_name=image_names, targetA=preds[:,0], targetB=preds[:,1], targetC=preds[:,2], targetD=preds[:,3], targetE=preds[:,4], targetF=preds[:,5], targetG=preds[:,6]))
submission = submission.sort_values('image_name')
submission.to_csv('cuswwith_TTA_submission.csv', index=False)
submission.head()

In [ ]:
len(submission)


In [ ]:
np.save('cuswfoldWisePredictions_Distilled_EFF_NETB2.npy', pred_foldWise)